# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.30.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'HealthInsuranceCrossSell'
project_folder = './crossSell-project'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RCFXNJ8TL to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
# TODO: Put your automl settings here
#automl_settings = {}

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "notebook147751"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().
key = "CrossSell Dataset"
description_text = "CrossSell from Kragle"
 
dataset = ws.datasets[key] 
df = dataset.to_pandas_dataframe()
df.describe()


# TODO: Put your automl config here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Response",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#automl_config = AutoMLConfig()

Found existing cluster, use it.

Running


In [4]:
dataset.take(5).to_pandas_dataframe()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,True,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,False,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,True,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,False,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,False,27496.0,152.0,39,0


In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
HealthInsuranceCrossSell,AutoML_f55bc43c-4d18-4915-b727-7265a7af51cc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Usin

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
remote_run.wait_for_completion()

metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

best_automl, best_model = remote_run.get_output()
print(best_model)

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/moun...
), random_state=0, tree_method='auto'))], verbose=False))], meta_learner=LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False, fit_intercept=True, intercept_scaling=1.0, l1_ratios=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', random_state=None, refit=True, scoring=<azureml.automl.runtime.stack_ensemble_base.Scorer object at 0x7f9bee83f588>, solver='lbfgs', tol=0.0001, verbose=0), training_cv_folds=5))],
         verbose=False)


In [8]:
import joblib, pickle
joblib.dump(best_model, 'best_model.pkl')

['best_model.pkl']

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

model = Model.register(workspace = ws, model_name = 'best_model_output', model_path = 'best_model.pkl')
print(model.name, model.id, model.version, sep='\t')

# Create the environment
env = best_automl.get_environment()
#conda_dep = CondaDependencies()

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "crosssell", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)



Registering model best_model_output
best_model_output	best_model_output:4	4
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-23 04:27:15+00:00 Creating Container Registry if not exists.
2021-06-23 04:27:15+00:00 Registering the environment.
2021-06-23 04:27:15+00:00 Use the existing image.
2021-06-23 04:27:16+00:00 Generating deployment configuration.
2021-06-23 04:27:16+00:00 Submitting deployment to compute.
2021-06-23 04:27:19+00:00 Checking the status of deployment crosssell..
2021-06-23 04:31:43+00:00 Checking the status of inference endpoint crosssell.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://72f17880-e4c7-4537-9b96-515ee04648dc.southcentralus.azurecontainer.io/score
http://72f17880-e4c7-4537-9b96-515ee04648dc.southcentralus.azurecontainer.io/swagger.json


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


TODO: In the cell below, send a request to the web service you deployed to test it.

In [14]:
import requests
import json

# URL for the web service
scoring_uri = 'http://72f17880-e4c7-4537-9b96-515ee04648dc.southcentralus.azurecontainer.io/score'

# Set the content type
# Set the content type
headers = {'Content-Type': 'application/json'}

# Two sets of data to score, so we get two results back
data = {"data":
        [{
                "id" :381110,
                "Gender": "Male",
                "Age" :25,
                "Driving_License":1,
                "Region_Code" :11,
                "Previously_Insured" :1,
                "Vehicle_Age" :"< 1 Year",
                "Vehicle_Damage":"No",
                "Annual_Premium" :35786,
                "Policy_Sales_Channel" :152,
                "Vintage" :53
                }
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.text)

Response Code :  200
Predicted Value :  [0]


TODO: In the cell below, print the logs of the web service and delete the service

In [15]:
print(service.get_logs())

2021-06-23T04:31:29,258237000+00:00 - iot-server/run 
2021-06-23T04:31:29,258814700+00:00 - rsyslog/run 
2021-06-23T04:31:29,263267100+00:00 - gunicorn/run 
rsyslogd: /azureml-envs/azureml_9f1639e55483048380059c02b396daba/lib/libuuid.so.1: no version information available (required by rsyslogd)
2021-06-23T04:31:29,321035300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_9f1639e55483048380059c02b396daba/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9f1639e55483048380059c02b396daba/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9f1639e55483048380059c02b396daba/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9f1639e55483048380059c02b396daba/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [ ]:
service.delete()